# Spark. Алгоритмы машинного обучения

Полный список алгоритмов можно посмотреть в официальной документации https://spark.apache.org/docs/latest/ml-classification-regression.html

In [ ]:
%%capture
!pip install pyspark

In [ ]:
# import pyspark
# from pyspark.sql import SparkSession
# # В качестве примера используется порт по умолчанию (local)
# conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
# # Развертывание среды Spark с указанными настройками
# sc = pyspark.SparkContext(conf=conf)
# # Запуск Spark-сессии
# spark = SparkSession(sc)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
import pandas as pd
import numpy as np
from sklearn import datasets

In [ ]:
iris = datasets.load_iris()
X = iris.data[:,:4]
y = iris.target
df = pd.DataFrame(X,columns=iris.feature_names)
df['target'] = y
df.columns = ['sepal_length','sepal_width','petal_length','petal_width','target']
df['name_iris'] = df['target'].apply(lambda _:iris.target_names[_])

In [ ]:
print(df.head())
print('---------------------------------------')
print(df.info())
print('---------------------------------------')
print(df.name_iris.value_counts(normalize=True))

   sepal_length  sepal_width  petal_length  petal_width  target name_iris
0           5.1          3.5           1.4          0.2       0    setosa
1           4.9          3.0           1.4          0.2       0    setosa
2           4.7          3.2           1.3          0.2       0    setosa
3           4.6          3.1           1.5          0.2       0    setosa
4           5.0          3.6           1.4          0.2       0    setosa
---------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   target        150 non-null    int64  
 5   name_iris     150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB
None
-

In [ ]:
df.to_csv('data.csv', index=False)

In [ ]:
df = spark.read.csv('data.csv',inferSchema=True,header=True)

In [ ]:
df.show(5)

+------------+-----------+------------+-----------+------+---------+
|sepal_length|sepal_width|petal_length|petal_width|target|name_iris|
+------------+-----------+------------+-----------+------+---------+
|         5.1|        3.5|         1.4|        0.2|     0|   setosa|
|         4.9|        3.0|         1.4|        0.2|     0|   setosa|
|         4.7|        3.2|         1.3|        0.2|     0|   setosa|
|         4.6|        3.1|         1.5|        0.2|     0|   setosa|
|         5.0|        3.6|         1.4|        0.2|     0|   setosa|
+------------+-----------+------------+-----------+------+---------+
only showing top 5 rows



In [ ]:
df.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- target: integer (nullable = true)
 |-- name_iris: string (nullable = true)



In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
df_assembler = VectorAssembler(inputCols=['sepal_length','sepal_width','petal_length','petal_width'],outputCol='features')

In [ ]:
df = df_assembler.transform(df)

In [ ]:
df.select(['features','target']).show(5,False)

+-----------------+------+
|features         |target|
+-----------------+------+
|[5.1,3.5,1.4,0.2]|0     |
|[4.9,3.0,1.4,0.2]|0     |
|[4.7,3.2,1.3,0.2]|0     |
|[4.6,3.1,1.5,0.2]|0     |
|[5.0,3.6,1.4,0.2]|0     |
+-----------------+------+
only showing top 5 rows



In [ ]:
model_df = df.select(['features','target'])

In [ ]:
model_df_2_class = model_df.filter(model_df.target.isin(0,1))

In [ ]:
training_df, test_df = model_df.randomSplit([0.75,0.25])

In [ ]:
training_df_2_class, test_df_2_class = model_df_2_class.randomSplit([0.75,0.25])

## GBT-алгоритм

GBT (Gradient Boost Tree) – это алгоритм машинного обучения, использующийся для задач классификации и регрессии, который строит модель предсказания величины в форме ансамбля слабых предсказывающих моделей, в частности деревьев решений.

In [ ]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
gbt = GBTClassifier(labelCol='target',featuresCol='features', maxDepth=10)

In [ ]:
model_gbt = gbt.fit(training_df_2_class)

In [ ]:
predictions_gbt = model_gbt.transform(test_df_2_class)

In [ ]:
predictions_gbt.show(5)

+-----------------+------+--------------------+--------------------+----------+
|         features|target|       rawPrediction|         probability|prediction|
+-----------------+------+--------------------+--------------------+----------+
|[4.4,2.9,1.4,0.2]|     0|[1.54350200272498...|[0.95635347857270...|       0.0|
|[4.4,3.2,1.3,0.2]|     0|[1.54350200272498...|[0.95635347857270...|       0.0|
|[4.6,3.1,1.5,0.2]|     0|[1.54350200272498...|[0.95635347857270...|       0.0|
|[4.6,3.4,1.4,0.3]|     0|[1.54350200272498...|[0.95635347857270...|       0.0|
|[4.6,3.6,1.0,0.2]|     0|[1.54350200272498...|[0.95635347857270...|       0.0|
+-----------------+------+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
evaluator = MulticlassClassificationEvaluator(
                                            labelCol="target", predictionCol="prediction", metricName="accuracy")
accuracy_gbt = evaluator.evaluate(predictions_gbt)
print("Test Error = %g" % (1.0 - accuracy_gbt))

Test Error = 0


## Алгоритм One-vs-Rest

Алгоритм «Один против всех» (One vs all или One vs Rest) – это модель, которая предоставляет путь решения бинарной классификации из нескольких возможных решений. В течение обучения модель проходит через последовательность бинарных классификаторов (по одному бинарному классификатору для каждого возможного выхода), тренируя каждый их них отвечать на отдельный классификационный вопрос.

In [ ]:
from pyspark.ml.classification import LogisticRegression, OneVsRest

In [ ]:
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

In [ ]:
ovr = OneVsRest(classifier=lr,labelCol='target',featuresCol='features')

In [ ]:
model_ovr = ovr.fit(training_df)
predictions_ovr = model_ovr.transform(test_df)

In [ ]:
evaluator = MulticlassClassificationEvaluator(
                                            labelCol="target", predictionCol="prediction", metricName="accuracy")
accuracy_ovr = evaluator.evaluate(predictions_ovr)
print("Test Error = %g" % (1.0 - accuracy_ovr))

Test Error = 0.0625


## Алгоритм случайного леса

Случайный лес (random forest) – это алгоритм машинного обучения, который заключается в использовании ансамбля (совокупности) деревьев решений (decision trees). Ключевая идея заключается в том, что качество классификации в случайном лесу повышается за счет большого количества ансамблей деревьев решений. Классификация проводится путем голосования деревьев, где каждое дерево относит классифицируемый объект к одному из классов. Побеждает тот класс, за который проголосовало наибольшее число деревьев. Оптимальное число деревьев подбирается таким образом, чтобы минимизировать ошибку классификации на тестовой выборке.

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(featuresCol="features", labelCol='target', numTrees=10)

In [ ]:
model_rf = rf.fit(training_df)
predictions_rf = model_rf.transform(test_df)

In [ ]:
evaluator = RegressionEvaluator(
                                labelCol="target", predictionCol="prediction", metricName="rmse")
rmse_rf = evaluator.evaluate(predictions_rf)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse_rf)

Root Mean Squared Error (RMSE) on test data = 0.177658


## Метод опорных векторов

Метод опорных векторов (support vector machine, SVM) — алгоритм классификации, в основе которого лежит определяемая разделяющая гиперплоскость (линия, прямая, многомерные плоскости). Другими словами, при заданных тренировочных данных алгоритм находит такую гиперплоскость, которая разделяет данные, принадлежащие разным классам, самым оптимальным способом. В двухмерном пространстве гиперплоскостью служит прямая линия.

In [ ]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
lsvc = LinearSVC(labelCol="target", regParam=0.1)
model_lsvc = lsvc.fit(training_df_2_class)

In [ ]:
print("Coefficients:", model_lsvc.coefficients)
print("Intercept:", model_lsvc.intercept)

Coefficients: [0.2825404213109134,-0.811639491578646,0.3446898161477943,0.9344901652684454]
Intercept: -0.6841886400260185


In [ ]:
predictions_lsvc = model_lsvc.transform(test_df_2_class)

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol="target")
accuracy_lsvc = evaluator.evaluate(predictions_lsvc)
print(f"Test Error =", (1.0 - accuracy_lsvc))

Test Error = 0.0


## Наивный байесовский классификатор

Наивный байесовский классификатор (Naive Bayes) — это алгоритм машинного обучения, предназначенный для многоклассовой классификации данных с независимыми признаками. За один проход вычисляется условная вероятность каждого признака, затем применяется теорема Байеса для нахождения распределения вероятности наблюдений.

Полиномиальный байесовский классификатор
Бернуллевский байесовский классификатор
Дополняющий байесовский классификатор
Гауссовский байесовский классификатор

In [ ]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(modelType="multinomial",labelCol='target')
model_nb = nb.fit(training_df)

In [ ]:
predictions_nb = model_nb.transform(test_df)
predictions_nb.show(5)

+-----------------+------+--------------------+--------------------+----------+
|         features|target|       rawPrediction|         probability|prediction|
+-----------------+------+--------------------+--------------------+----------+
|[4.3,3.0,1.1,0.1]|     0|[-9.9124117797173...|[0.72859812471276...|       0.0|
|[4.4,3.0,1.3,0.2]|     0|[-10.731713854480...|[0.67350355476710...|       0.0|
|[4.5,2.3,1.3,0.3]|     0|[-10.402464629198...|[0.54874547297232...|       0.0|
|[4.8,3.4,1.9,0.2]|     0|[-12.608071888050...|[0.64384364302941...|       0.0|
|[5.0,2.0,3.5,1.0]|     1|[-17.210413572404...|[0.07707022754093...|       1.0|
+-----------------+------+--------------------+--------------------+----------+
only showing top 5 rows



In [ ]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="target", 
    predictionCol="prediction",
    metricName="accuracy"
)
accuracy_nb = evaluator.evaluate(predictions_nb)
print(f"Test Error =", (1.0 - accuracy_nb))

Test Error = 0.0625


## Градиентный бустинг

Градиентный бустинг, он же Gradient Boosting, Gradient Boosted-Tree, более правильно будет назвать ансамбль деревьев решений, обученный с использованием градиентного бустинга.

Градиентный бустинг представляет собой ансамбль деревьев решений. В основе данного алгоритма лежит итеративное обучение деревьев решений с целью минимизировать функцию потерь. Благодаря особенностям деревьев решений градиентный бустинг способен работать с категориальными признаками , справляться с нелинейностями.

В Spark Ml градиентный бустинг поддерживает бинарную классификацию и регрессию с использованием как непрерывных признаков, так и категориальных. 

Gradient-Boosted Trees не поддерживает многоклассовую классификацию. Если вам она нужна, то используйте случайный лес (Random Forest).

В Spark Ml для градиентного бустинга предусмотрены три функции потерь: для классификации - Логистическая потеря (log loss) и для регрессии - Квадратическая ошибка и Абсолютная ошибка. 

In [ ]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.regression import GBTRegressor

## Деревья решений

Деревья решений (Decision trees) являются одним из самых популярных алгоритмов машинного обучения и используются для задач классификации (бинарной и многоклассовой) и регрессии. Деревья решений простоты, понятны, они хорошо обрабатывают категориальные значения, а также могут находить нелинейные связи. 

## Логистическая регрессия

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, labelCol='target')

In [ ]:
model_lr = lr.fit(training_df)

In [ ]:
print("Coefficients: \n" + str(model_lr.coefficientMatrix))
print("Intercept: " + str(model_lr.interceptVector))

Coefficients: 
3 X 4 CSRMatrix
(0,2) -0.2626
(0,3) -0.3154
(1,3) 0.2256
Intercept: [0.9335770202608067,-0.2726324816200717,-0.6609445386407349]


In [ ]:
predictions_lr = model_lr.transform(test_df)

In [ ]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="target", 
    predictionCol="prediction",
    metricName="accuracy"
)
accuracy_lr = evaluator.evaluate(predictions_lr)
print(f"Test Error =", (1.0 - accuracy_lr))

Test Error = 0.0
